In [ ]:
from google.colab.patches import cv2_imshow
import albumentations as A
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
import random
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Image Preprocessing**

In [ ]:
input_dir = '/content/drive/MyDrive/Melanoma Detection Folder/data/val/malignant'
output_dir = '/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Val/Malignant'

for filename in os.listdir(input_dir):
  if filename.endswith(".jpg") or filename.endswith(".jpeg"):
    # Load the image
    img_path = os.path.join(input_dir, filename)
    img = cv2.imread(img_path) #into a NumPy array

    #REMOVE HAIR
    # Gray scale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Black hat filter
    kernel = cv2.getStructuringElement(1, (9, 9)) #1 specify shape of SE and for 9*9
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # Gaussian filter (hair removal & noise)
    bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
    # Binary thresholding (MASK)
    ret, mask = cv2.threshold(bhg, 10, 255, cv2.THRESH_BINARY)
    # Replace pixels of the mask
    dst = cv2.inpaint(img, mask, 6, cv2.INPAINT_TELEA)

    # Apply Median filtering
    median = cv2.medianBlur(dst, 5)
    # Adjust the brightness and contrast
    # Adjusts the brightness by adding 10 to each pixel value
    brightness = 1
    # Adjusts the contrast by scaling the pixel values by 2.3
    contrast = 1.7
    image2 = cv2.addWeighted(median, contrast, np.zeros(median.shape, median.dtype), 0, brightness)

    #Laplacian apllied
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    # Sharpen the image
    sharpened_image = cv2.filter2D(image2, -1, kernel)

    # Save the processed image to the output directory
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, sharpened_image)

Delete content

In [ ]:
for filename in os.listdir('/content/drive/MyDrive/DMTProj/archive/Processed/val/Benign'):
    file_path = os.path.join('/content/drive/MyDrive/DMTProj/archive/Processed/val/Benign', filename)
    try:
        # remove the file
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Error deleting file: {e}")

**Count files**


In [ ]:
dir_path = '/content/drive/MyDrive/Melanoma Detection Folder/Preprocessed Dataset/Val/Malignant'
num_files = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
print("Number of files in directory:", num_files)

Number of files in directory: 300
